<h1 style="text-align: center;"> Nationwide Sponsored Data Science Workshop </h1>


A github repository containing the dataset, data dictionary, notebook, and scripts for installing all required dependencies can be found [here](https://github.com/QuinnMLr/Nationwide-ML-Workshop) 

In this workshop, we will be helping a hypothetical insurance company, ACME Insurance, figure out what drives customer satisfaction using data obtained in a customer survey ___(NOTE: THIS IS NOT REAL DATA)___.  Understanding how various factors such as age, region, and premium changes influence customer satisfaction helps companies like ACME make informed decisions that may increase satisfaction, and hence retain more customers.

This is beginner-intermediate level and does not assume any prior data science experience 

The notebook demonstrates the following fundamental data science concepts:
- Loading data into a notebook
- Handling duplicate data
- Handling null values
- Mapping variable values
- Converting data types
- Visualizing distributions
- Visualizing correlations 
- Handling date-time data
- Creating dummy variables
- Splitting data into test and training sets
- Creating a regression model
- Creating a random forest 
- Comparing models

___We'll begin with importing the libraries we'll use throughout the notebook and configure some display settings.  Then, we'll load in the data, glimpse it, view the metadata, and look for missing values.___

In [ ]:
#Import libraries used throughout notebook
import pandas as pd
import numpy as np

In [ ]:
#Tell pandas to display only 3 decimals in tables
pd.options.display.float_format = '{:,.8f}'.format

#Show all columns with head()
pd.set_option('display.max_columns', None)

#Read in dataset
data_path = 'ACME_SURVEY.csv'
acme = pd.read_csv(data_path)

#Glimpse data
acme.head(25)

In [ ]:
#View metadata
acme.info()

In [ ]:
#View missing values
acme.isnull().sum()

___Let's decide how we're going to leverage our data to answer the question 'What drives customer satisfaction?'  Notice 'Q1' to 'Q6' are some form of satisfaction rating on a 10-point scale?  Let's see if these ratings are correlated___

In [ ]:
import seaborn as sns

#Take columns 'Q1' to 'Q6'
acme_q = acme.loc[:, 'Q1':'Q6']

#Drop NaN's
acme_q3 = acme_q.dropna()

#Create correlation matrix
corr = acme_q3.corr()

#Create correlation matrix heatmap
sns.heatmap(corr, cmap ="YlGnBu", annot = True,vmin = 0,vmax = 1)

___Looks like there is strong correlation.  Because these variables are strongly correlated, and there're quite a bit of missing values, we'll take the average of the ratings to be our response variable___

In [ ]:
#Create average satisfaction variable
acme['SAT_AVE'] = acme[['Q1','Q2','Q3','Q4','Q5','Q6']].mean(axis=1)

acme['SAT_AVE'].head()

In [ ]:
#view summary statistics of response variable
summary = acme['SAT_AVE'].describe()
summary

___Now let's clean up the data.  This will involve handling duplicate data, renaming variables, remapping values of variables, converting data types and creating derived variables to arrive at the dataset to be used in modeling.___

In [ ]:
#Get number of rows
acme['RESP_ID'].size

In [ ]:
#Get number of unique rows
acme['RESP_ID'].unique().size

In [ ]:
#Find duplicate rows
duplicateRowsDF = acme[acme.duplicated(['RESP_ID'],keep=False)]

#Display rows
duplicateRowsDF.head()

In [ ]:
#remove duplicate row
acme.drop_duplicates(subset ='RESP_ID', keep = 'first', inplace = True)

#Ensure duplicate is removed
acme['RESP_ID'].size

In [ ]:
#Display values counts of 'SCREENER_1'
acme['SCREENER_1'].value_counts()

In [ ]:
#Replace all NaN values with 0
acme['SCREENER_1'] = acme['SCREENER_1'].fillna('0')

#Map values to integers
acme['CLAIM'] = acme['SCREENER_1'].replace({'No':'0','DK':'0','Yes':'1'}).astype('category')

#Display values counts
acme['CLAIM'].value_counts()

In [ ]:
#Display values counts of 'SCREENER_3'
acme['SCREENER_3'].value_counts()

In [ ]:
#Replace all NaN values with 0
acme['SCREENER_3'] = acme['SCREENER_3'].fillna('0')

#Map values to integers
acme['PREMIUM'] = acme['SCREENER_3'].replace({'No':'0',"DON'T KNOW":'0','Yes':'1'}).astype('category')

#Display value counts
acme['PREMIUM'].value_counts()

In [ ]:
#Display value counts of 'POLICY_TYPE'
acme.groupby('POLICY_TYPE').size()

In [ ]:
#Convert 'POLICY_TYPE' to categorical variable
acme['POLICY_TYPE'] = acme['POLICY_TYPE'].astype('category')

In [ ]:
#Rename 'SCREENER_2' to TENURE
acme['TENURE'] = acme['SCREENER_2']

#View 'TENURE' summary statistics
summary = acme['TENURE'].describe()
summary

In [ ]:
#View density plot of 'TENURE'
acme['TENURE'].plot.kde()

In [ ]:
#Take columns 'Q17_1' to 'Q17_8'
acme_tp = acme.loc[:, 'Q7_1':'Q7_8']

acme_tp.head()

In [ ]:
#Convert NaN's to string
acme_tp = acme_tp.astype(str)

#View all values of 'Q17_1' to 'Q17_8'
np.unique(acme_tp.values)

In [ ]:
#Replace all NaN values with 0
acme['Q7_1'] = acme['Q7_1'].fillna('0')
acme['Q7_2'] = acme['Q7_2'].fillna('0')
acme['Q7_3'] = acme['Q7_3'].fillna('0')
acme['Q7_4'] = acme['Q7_4'].fillna('0')
acme['Q7_5'] = acme['Q7_5'].fillna('0')
acme['Q7_6'] = acme['Q7_6'].fillna('0')
acme['Q7_7'] = acme['Q7_7'].fillna('0')
acme['Q7_8'] = acme['Q7_8'].fillna('0')

#Map 'Yes' to 1 and all other values to 0
map_contact_num = {'No':'0','DK':'0','Yes':'1'}

#Apply mapping and convert values to numeric type
acme['Q7_1'] = pd.to_numeric(acme['Q7_1'].replace(map_contact_num))
acme['Q7_2'] = pd.to_numeric(acme['Q7_2'].replace(map_contact_num))
acme['Q7_3'] = pd.to_numeric(acme['Q7_3'].replace(map_contact_num))
acme['Q7_4'] = pd.to_numeric(acme['Q7_4'].replace(map_contact_num))
acme['Q7_5'] = pd.to_numeric(acme['Q7_5'].replace(map_contact_num))
acme['Q7_6'] = pd.to_numeric(acme['Q7_6'].replace(map_contact_num))
acme['Q7_7'] = pd.to_numeric(acme['Q7_7'].replace(map_contact_num))
acme['Q7_8'] = pd.to_numeric(acme['Q7_8'].replace(map_contact_num))



#Calculate the number of times each customer was in contact with an agent about a claim
acme['Q7_SUM'] = acme.loc[:,'Q7_1':'Q7_8'].sum(axis=1)

#Display value counts 
acme['Q7_SUM'].value_counts()

In [ ]:
acme['Q7_SUM'].hist(bins=7)

In [ ]:
#View start/end time
acme[['SURVEY_START_TIME','SURVEY_END_TIME']].head()

In [ ]:
import datetime

#Convert 'SURVEY_START_TIME' and 'SURVEY_START_TIME' to datetimes
acme['SURVEY_START_TIME'] = pd.to_datetime(acme['SURVEY_START_TIME'])
acme['SURVEY_END_TIME'] = pd.to_datetime(acme['SURVEY_END_TIME'])

#Get duration of call
acme['SURVEY_DURATION'] = acme['SURVEY_END_TIME']-acme['SURVEY_START_TIME']

#Convert to seconds
acme['SURVEY_DURATION'] = acme['SURVEY_DURATION'].astype('timedelta64[s]')

acme['SURVEY_DURATION'].head()

In [ ]:
#Display value counts of 'GENERATION'
acme.groupby('GENERATION').size()

In [ ]:
from pandas.api.types import CategoricalDtype

#Convert 'GENERATION' to ordered categorical variable
acme['GENERATION'] = acme['GENERATION'].astype('category')

#Check data type
acme['GENERATION'].dtype

In [ ]:
#Display value counts of 'REGION'
acme.groupby('REGION').size()

In [ ]:
#Convert 'REGION' to ordered categorical variable
acme['REGION'] = acme['REGION'].astype('category')

#Check data type
acme['REGION'].dtype

In [ ]:
#Take variables to be used in analysis
acme_final_variables = acme[["REGION", "GENERATION", "POLICY_TYPE", "TENURE", 
                    "CLAIM", "PREMIUM", "SURVEY_DURATION", "Q7_SUM", "SAT_AVE"]]

#look over final dataset
acme_final_variables.head()

In [ ]:
#look over final dataset metadata
acme_final_variables.info()

___We will use one-hot encoding to transform our categorical variables to a form our models will understand.___

In [ ]:
#Create dummy variables for categorical variables
acme_final = pd.get_dummies(acme_final_variables, prefix=['REGION','GENERATION','POLICY_TYPE','PREMIUM','CLAIM'])

#Drop reference variables to avoid multicolinearity
acme_final = acme_final.drop(['REGION_Mid-Atlantic','GENERATION_Baby Boomers','POLICY_TYPE_Multi-Line','CLAIM_0','PREMIUM_0'],axis='columns')

acme_final.head()

___Finally, we are ready to build and interpret our models.  We will be creating and interpretting a regression model and a random forest model.  We will compare how well they are able to make predictions on a testing set, and the degree of influence each of the predictor variables had on the response___

## Multiple Regression

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


#Create response variable dataframe
y = acme_final['SAT_AVE']

#Create feature variables dataframe
X_1 = acme_final.loc[:, 'TENURE':'Q7_SUM']

X_2 = acme_final.loc[:, 'REGION_California':'CLAIM_1']

X = pd.concat([X_1,X_2], axis=1)

#Split data into training and testing sets (training: 75%, Testing: 25%)
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.25, random_state = 42)

#Create OLS multiple regressions model
reg_model = sm.OLS(endog=train_labels, exog=sm.add_constant(train_features)).fit()
reg_model.summary()

In [ ]:
from sklearn.metrics import mean_squared_error

#Use regression model to predict on testing data
reg_preds = reg_model.predict(sm.add_constant(test_features))

#Calculate the average squared difference between the estimated values and the actual value
MSE = mean_squared_error(y_true = test_labels, y_pred = reg_preds)

#Print MSE
print('Mean Squared Error:', MSE)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
RFR = RandomForestRegressor(n_estimators = 1500,max_depth=4,random_state = 42)

# Train the model on training data
rf = RFR.fit(train_features, train_labels)

In [ ]:
# Use the forest to predict on test data
rf_preds = rf.predict(test_features)

#Calculate the average squared difference between the estimated values and the actual value
MSE = mean_squared_error(y_true = test_labels, y_pred = rf_preds)

#Print MSE
print('Mean Squared Error:', MSE)

In [ ]:
#View features' importances in determining splits
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = train_features.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances